In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import mlflow
import pickle
from tqdm import tqdm

# Experiment setup

In [2]:
mlflow.set_tracking_uri('../mlflow')

if mlflow.get_experiment_by_name('image-based recsys') is None:
    mlflow.set_experiment('image-based recsys')

# Data

In [3]:
with open('../data/styles-44c73c23.pkl', 'rb') as f:
    styles = pickle.load(f)

In [4]:
embeddings = pd.read_parquet('../data/embeddings-44c73c23.parquet')

In [5]:
styles.shape, embeddings.shape

((44412, 40), (44412, 1281))

In [6]:
embeddings = embeddings.set_index('image')
styles = styles.set_index('id')

/home/zeinovich/anaconda3/envs/image-ranking/lib/python3.9/site-packages/pandas/core/indexes/base.py:6999: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  return Index(sequences[0], name=names)


In [7]:
styles.index.isin(embeddings.index).sum(), embeddings.index.isin(styles.index).sum()

(44412, 44412)

In [8]:
holdout = pd.read_csv('../data/holdout_ids.csv')
holdout.head()

,id
0,23591
1,39604
2,49462
3,26809
4,56261


In [9]:
styles = styles[~styles.index.isin(holdout['id'])]
embeddings = embeddings[~embeddings.index.isin(holdout['id'])]

In [10]:
styles = styles.sort_index()
embeddings = embeddings.sort_index()

# Scaling

In [11]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(embeddings)
embeddings = pd.DataFrame(scaler.fit_transform(embeddings.values), columns=embeddings.columns, index=embeddings.index)

In [12]:
embeddings

,0,1,2,3,4,5,6,7,8,9,...,1270,1271,1272,1273,1274,1275,1276,1277,1278,1279
image,,,,,,,,,,,,,,,,,,,,,
1163,-0.559129,0.064624,-0.541653,0.494476,-0.732576,-0.364267,0.217243,-0.063711,-0.646313,-0.179434,...,-0.627197,0.065260,0.076759,0.114998,1.375037,2.669481,-0.539308,1.235730,-0.264875,-0.551288
1164,-0.766388,0.337273,-0.494332,0.815344,-0.737816,-0.310305,0.393708,0.110811,-0.465037,0.744875,...,-0.638863,-0.071575,0.001753,0.326641,1.232199,2.403879,-0.615210,1.178555,-0.179312,-0.542332
1165,0.001387,-0.152014,-0.213541,0.618311,-0.020924,-0.840245,-1.184028,-0.156743,-0.092964,-1.446525,...,0.190283,-0.624775,1.240647,-0.261998,-0.666391,-0.848931,-0.066920,1.264973,-0.255418,-0.544567
1525,1.538574,0.237531,-0.865232,-0.432264,-0.699365,-0.021174,0.358182,0.011606,-0.859797,1.771345,...,-0.424415,-0.100269,-1.089163,0.107877,-0.042046,-0.429806,0.405103,-0.404875,1.598210,-0.669227
1526,0.989720,0.304483,-0.677383,-0.325525,-0.689746,-0.702989,0.339682,-0.302013,-0.595768,0.823138,...,-0.563310,-1.299171,-0.895376,0.419114,0.449609,-0.270480,1.217311,-0.358990,-0.562421,-0.664526
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59993,-0.004618,0.085327,-0.388462,0.872409,-0.118179,-1.221492,-1.003921,0.093502,-0.489033,-0.973793,...,-0.558295,-1.097023,0.531259,0.379813,-0.958839,-0.379470,0.532411,0.548971,0.065494,-0.673705
59995,-0.785797,0.232165,-0.486595,0.751630,0.623703,0.063441,-0.986461,-0.007501,-0.640681,-0.456576,...,-0.697831,-1.554046,0.800550,-0.643488,-0.454050,-0.250345,0.117627,1.436240,-0.149531,-0.489758
59996,0.181431,0.302661,-0.202702,1.028051,0.507657,-0.647934,-0.531728,-0.185867,-0.390892,-1.455807,...,0.173168,-1.495853,0.527829,0.478845,-1.151165,1.128537,0.311293,1.293028,0.076110,-0.195862


# Nearest Neighbors

In [24]:
from sklearn.model_selection import StratifiedKFold
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import precision_score, recall_score

def get_recall(y_true, y_pred, k=5):
    y_true = np.array(y_true.values)
    y_pred = np.array(y_pred.values)
    y_true = np.repeat(y_true, k)

    return recall_score(y_true.reshape(-1), y_pred.reshape(-1), average='weighted')

def get_precision(y_true, y_pred, k=5):
    y_true = np.array(y_true.values)
    y_pred = np.array(y_pred.values)
    y_true = np.repeat(y_true, k)

    return precision_score(y_true.reshape(-1), y_pred.reshape(-1), average='weighted')

def get_knn_score(X, y, k=5, metric='cosine', category='sub'):
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    prec = []
    recall = []
    # mlflow.set_experiment('image-based recsys')
    with mlflow.start_run(experiment_id=mlflow.get_experiment_by_name('image-based recsys').experiment_id):
        
        mlflow.log_param('k', k)
        mlflow.log_param('category', category)
        mlflow.log_param('embeddings', 'EfficientNet_V2_S')
        mlflow.log_param('dataset_hash', '44c73c23')
        mlflow.log_param('metric', metric)
        
        for train_index, test_index in (progress := tqdm(skf.split(X, y), total=5, miniters=1)):
            progress.set_description(f'{metric=}')
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]

            knn = NearestNeighbors(n_neighbors=k, metric=metric)
            knn.fit(X_train)

            _, indices = knn.kneighbors(X_test)

            y_pred = y_train.iloc[indices.reshape(-1)]
            prec.append(get_precision(y_test, y_pred, k))
            recall.append(get_recall(y_test, y_pred, k))

        mlflow.log_metric(f'precision-at-{k}', np.mean(prec))
        mlflow.log_metric(f'recall-at-{k}', np.mean(recall))

    return {'precision': np.mean(prec), 'recall': np.mean(recall)}

In [25]:
K = 5
METRICS = ['cosine', 'euclidean']

scores = []

for metric in METRICS:
    score = get_knn_score(embeddings, styles['masterCategory'].cat.codes, k=K, metric=metric, category='master')
    scores.append(score)

scores = pd.DataFrame({'metric': METRICS, 'score': scores})
scores

metric='euclidean': 100%|█████████████████████████████████████████████████████████████████| 5/5 [00:53<00:00, 10.63s/it]


,metric,score
0,cosine,"{'precision': 0.9908139683533242, 'recall': 0...."
1,euclidean,"{'precision': 0.9902476837910734, 'recall': 0...."


In [26]:
K = 5
METRICS = ['cosine', 'euclidean']

scores = []

for metric in METRICS:
    score = get_knn_score(embeddings, styles['subCategory'].cat.codes, k=K, metric=metric, category='sub')
    scores.append(score)

scores = pd.DataFrame({'metric': METRICS, 'score': scores})
scores

metric='euclidean': 100%|█████████████████████████████████████████████████████████████████| 5/5 [01:34<00:00, 19.00s/it]


,metric,score
0,cosine,"{'precision': 0.9334580175735366, 'recall': 0...."
1,euclidean,"{'precision': 0.9310988692484546, 'recall': 0...."


In [15]:
knn = NearestNeighbors(n_neighbors=K, metric='euclidean')
knn.fit(embeddings)

NearestNeighbors(metric='euclidean')

In [16]:
s = styles.sample(1).index

sample_emb = embeddings.loc[s]

In [17]:
s

Int64Index([55002], dtype='int64', name='id')

In [18]:
pd.set_option('display.max_columns', 100)

In [19]:
styles.loc[s]

,price,discountedPrice,styleType,productTypeId,articleNumber,visualTag,productDisplayName,variantName,myntraRating,catalogAddDate,brandName,ageGroup,gender,baseColour,colour1,colour2,fashionType,season,year,usage,vat,displayCategories,weight,navigationId,landingPageUrl,articleAttributes,brandUserProfile,codEnabled,styleImages,lookGoodAlbum,style360Images,masterCategory,subCategory,articleType,isEMIEnabled,productDescriptors,styleOptions,colours,discountData
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
55002,575,517,P,467,TRLF100,,Lakme Absolute Liplast Day Passion Lip Gloss,NW LAK ABS LIPLAST 16H DAY PASSION 1.8ML,0,1454675734,Lakme,Adults-Women,Women,Purple,NA,NA,Core,Spring,2017,NA,14.5,NaN,0,0,Lip-Gloss/Lakme/Lakme-Absolute-Liplast-Day-Pas...,"{'Colour Shade Name': 'Passion', 'Units Per Bu...",{'uidx': 'f19314de.3f2e.489f.884d.074ca41d9e24...,True,{'default': {'imageURL': 'http://assets.myntas...,{},{},Personal Care,Lips,Lip Gloss,False,lakme Absolute Lip Last comes with a unique br...,"[{'id': 207398, 'name': 'Size', 'value': '1.8m...",NaN,"{'discountType': 1, 'discountAmount': 58, 'dis..."


In [20]:
print(styles.loc[s, 'styleImages'].iloc[0]['default']['imageURL'])

http://assets.myntassets.com/v1/images/style/properties/Lakme-Absolute-Liplast-Day-Passion-Lip-Gloss_e6006bd50916d2ceb0ed45c0abd0dfcb_images.jpg


In [21]:
_, indices = knn.kneighbors(sample_emb)
indices

array([[32383, 32385, 32382, 32384, 32381]])

In [22]:
for i in indices[0]:
    print(styles.iloc[i]['styleImages']['default']['imageURL'])

http://assets.myntassets.com/assets/images/55000/2018/3/9/11520591673411-Lakme-Absolute-Liplast-Day-Rose-Lip-Gloss-9641520591673383-1.jpg
http://assets.myntassets.com/v1/images/style/properties/Lakme-Absolute-Liplast-Day-Passion-Lip-Gloss_e6006bd50916d2ceb0ed45c0abd0dfcb_images.jpg
http://assets.myntassets.com/assets/images/54997/2018/3/9/11520591661297-Lakme-Absolute-Liplast-Night-Silk-Lip-Gloss-5721520591661269-1.jpg
http://assets.myntassets.com/assets/images/55001/2018/3/9/11520591701090-Lakme-Absolute-Lip-Last-Day-Kiss-Lip-Colour-9831520591701048-1.jpg
http://assets.myntassets.com/v1/images/style/properties/Lakme-Absolute-Liplast-Night-Glam-Lip-Gloss_2e3002a88e9a488c1fc52825676fbd5d_images.jpg
